In [2]:
import os.path

import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix


In [3]:
import spacy

In [4]:
#df_sample = pd.read_csv('data.csv')

In [5]:
train=pd.read_csv('train.csv')

In [6]:
train.head()

,Unnamed: 0,text,target
0,156512,"There is a saying in my home that goes, ""You t...",0
1,156513,Williams-Sonoma: Strategies and Future Prospec...,0
2,156514,"Shristi, your first two paragraphs are irrelev...",0
3,156515,Extended Lifespan and Its Great Danger Essay\n...,0
4,156516,Imperialism and Nationalism in Middle Eastern ...,0


In [7]:
df_train_idx2graph=pd.read_csv('train_idx2graph.csv').reset_index(drop=True)

In [8]:
df_train_idx2graph=df_train_idx2graph['1']

In [9]:
df_train_idx2graph.head()

0    [[  0   1   1 ... 297 297 298]\n [  1   0   3 ...
1    [[   0    1    2 ... 1105 1105 1106]\n [   2  ...
2    [[  0   0   0   1   2   3   4   5   5   5   5 ...
3    [[   0    1    1 ... 1435 1436 1437]\n [   1  ...
4    [[   0    0    0 ... 1643 1643 1644]\n [   1  ...
Name: 1, dtype: object

In [10]:
train_idx2graph=df_train_idx2graph.to_numpy()

In [11]:
train_idx2graph

array(['[[  0   1   1 ... 297 297 298]\n [  1   0   3 ... 295 296 281]]',
       '[[   0    1    2 ... 1105 1105 1106]\n [   2    2    0 ... 1103 1104 1101]]',
       '[[  0   0   0   1   2   3   4   5   5   5   5   6   6   6   7   7   8   8\n    9  10  11  11  11  12  13  14  14  14  15  15  16  17  17  18  19  19\n   19  19  19  20  21  22  22  22  22  23  23  24  25  26  26  26  26  27\n   28  29  30  30  30  30  31  32  32  32  32  32  33  34  34  35  36  36\n   36  37  38  39  40  41  42  42  42  42  42  42  42  42  42  42  43  44\n   44  44  45  45  46  46  47  47  48  49  49  50  51  52  52  52  52  53\n   54  54  55  56  57  58  59  60  60  60  60  61  61  62  63  63  64  64\n   64  65  66  67  67  68  69  70  70  70  70  70  71  72  72  72  73  74\n   75  75  76  77  77  77  77  77  78  78  79  80  81  81  82  83  83  83\n   83  84  85  86  87  88  88  88  88  88  88  88  89  90  91  91  92  92\n   93  94  94  95  96  96  97  97  98  99  99  99 100 101 102 102 102 103\n  104 1

# balanced train and test data in each df

In [ ]:
ratio=0.8
df_human= df_sample[df_sample['source']=='Human']
df_llm = df_sample[df_sample['source']!='Human']
df_llm.source='LLM'
cut_idx_l=int(np.ceil(len(df_llm)*ratio))
cut_idx_h=int(np.ceil(len(df_human)*ratio))
df_train_human = df_human[:cut_idx_h]
df_test_human = df_human[cut_idx_h:]
df_train_llm = df_llm[:cut_idx_l]
df_test_llm = df_llm[cut_idx_l:]
df_train=pd.concat([df_train_human,df_train_llm])
df_test=pd.concat([df_test_human,df_test_llm])

In [ ]:
df_train.source.value_counts()


In [ ]:

df_test.source.value_counts()

In [ ]:
df_train.head(10)

In [ ]:
df_train.shape

In [ ]:
train

In [ ]:
df_train.isna().sum()

In [ ]:
df_test.shape

In [ ]:
df_test.isna().sum()

In [ ]:
!python3 -m spacy download en_core_web_lg

In [ ]:
nlp = spacy.load("en_core_web_lg")
doc = nlp("Apple shares rose on the news. Apple pie is delicious.").vector

In [ ]:
doc

### do not forget to define labels vector

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()
df_train['target']=le.fit_transform(df_train['source'])
df_train.target.value_counts()

In [ ]:
df_test['target']=le.transform(df_test['source'])
df_test.target.value_counts()

In [ ]:
df_train=df_train.drop('source', axis=1)
df_test=df_test.drop('source',axis=1)

In [ ]:
df_train.info()

In [ ]:
df_test.info()

In [11]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${torch.__version__}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${torch.__version__}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

#from torch_geometric

2.1.2+cu121
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)


In [12]:
df_train=train


In [13]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 434666 entries, 0 to 434665
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  434666 non-null  int64 
 1   text        434666 non-null  object
 2   target      434666 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 9.9+ MB


In [14]:
df_train=df_train.drop(['Unnamed: 0'], axis =1)

In [15]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 434666 entries, 0 to 434665
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    434666 non-null  object
 1   target  434666 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 6.6+ MB


In [16]:
#df_train = pd.read_csv('train.csv', sep=',')

In [17]:
class Tokenizer(object):
    def __init__(self, word2idx=None, nlp_model="en_core_web_lg"):
        self.nlp= spacy.load(nlp_model)
        #self.word2idx=word2idx
        #self.idx=len(word2idx)
        if word2idx is None:
            self.word2idx = {}
            self.idx2word = {}
            self.idx = 0
            self.word2idx['<pad>'] = self.idx
            self.idx2word[self.idx] = '<pad>'
            self.idx += 1
            self.word2idx['<unk>'] = self.idx
            self.idx2word[self.idx] = 'unk'
            self.idx += 1
        else:
            self.word2idx = word2idx
            self.idx2word = { v:k for k,v in word2idx.items()}
        
    def fit_on_doc(self, doc:spacy.tokens.doc.Doc):
        for word in doc:
            word= str(word).lower()
            if word not in self.word2idx:
                self.word2idx[word]=self.idx
                self.idx2word[self.idx]=word
                self.idx += 1
    def text_to_doc(self, text):
        return self.nlp(text)
    def doc_to_sequence(self, doc:spacy.tokens.doc.Doc):
        sequence = []
        for w in doc:
            w = str(w).lower()
            word_id = self.word2idx.get(w,-1)
            if word_id == -1:
                word_id = self.word2idx['<unk>']
            sequence.append(word_id)
        if len(sequence) == 0:
            sequence = [0]
        return np.array(sequence, dtype=np.int32)
    def doc_to_adj(self, doc: spacy.tokens.doc.Doc):
        matrix = np.zeros((len(doc),len(doc))).astype('int32')
        for token in doc:
            for child in token.children:
                matrix[token.i][child.i] = 1
                matrix[child.i][token.i] = 1
        return matrix

In [18]:
tokenizer = Tokenizer()

In [19]:
tokenizer.word2idx

{'<pad>': 0, '<unk>': 1}

In [20]:
from tqdm import tqdm

In [21]:
df_train.isna().sum()

text      0
target    0
dtype: int64

In [22]:
df_train=df_train[:434666]

In [23]:
len(df_train)

434666

In [ ]:
train_idx2graph = {}

for i in tqdm(range(len(df_train))):
    text = df_train.text[i].lower().replace("/n", "").strip()
    doc = tokenizer.text_to_doc(text)
    tokenizer.fit_on_doc(doc)
    adj_matrix = tokenizer.doc_to_adj(doc)
    coo = coo_matrix(adj_matrix)
    train_idx2graph[i] = np.array([coo.row, coo.col])

In [24]:
len(train_idx2graph)

434666

In [ ]:
test_idx2graph = {}
for i in tqdm(range(len(df_test))):
    #doc=pd.concat([[df_train.text_tkn[i]],[df_test.prompt_tkn[i]])
    tokenizer.fit_on_doc(doc)
    adj_matrix = tokenizer.doc_to_adj(doc)
    coo = coo_matrix(adj_matrix)
    test_idx2graph[i] = np.array([coo.row, coo.col], dtype=np.int32)

In [ ]:
df_train.to_csv('train.csv')

In [ ]:
df_test.to_csv('test.csv')

In [ ]:
result=train_idx2graph.items()
da = list(result)
nparr=np.array(da)
train = pd.DataFrame(nparr)

In [ ]:
train.head()

In [ ]:
train.to_csv('train_idx2graph.csv')

In [ ]:
df_idx2graph=pd.read_csv('train_idx2graph.csv')

In [ ]:
np.loadtxt('train_idx2graph.csv', usecols=2)

In [ ]:
train_idx2graph=df_idx2graph['1']

In [ ]:
train_idx2graph.to_numpy(dtype='float')

In [ ]:
result=test_idx2graph.items()
da = list(result)
nparr=np.array(da)
test = pd.DataFrame(nparr)

In [ ]:
test.to_csv('test_idx2graph.csv')

In [ ]:
df_test_idx2graph=pd.read_csv('test_idx2graph.csv')

In [ ]:
np_test_idx2graph=df_test_idx2graph.to_numpy()

In [25]:
def load_word_vec(path, word2idx=None, embed_dim=300):
    fin = open(path, 'r', encoding='utf8', newline='\n', errors='ignore')
    word_vec = {}
    for line in fin:
        tokens = line.rstrip().split()
        word, vec = ' '.join(tokens[:-embed_dim]), tokens[-embed_dim:]
        if word in word2idx.keys():
            word_vec[word] = np.array(vec, dtype=np.float32)
    return word_vec

In [26]:
from torchtext.vocab import GloVe
def build_embedding_matrix(word2idx, embed_dim=300):
    embedding_matrix = np.zeros((len(word2idx), embed_dim))
    embedding_matrix[1, :] = np.random.uniform(-1/np.sqrt(embed_dim), 1/np.sqrt(embed_dim), (1, embed_dim))

    glob_vector='./glove.840B.300d.txt'
    word_vec = load_word_vec(glob_vector, word2idx=word2idx, embed_dim=embed_dim)

    for word, i in word2idx.items():
        vec = word_vec.get(word)
        if vec is not None:
            embedding_matrix[i] = vec
    return embedding_matrix

In [27]:
embedding_matrix = build_embedding_matrix(tokenizer.word2idx, 300)

In [28]:
print(embedding_matrix)

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000

In [29]:
from torch_geometric.data import InMemoryDataset, Data


class TrainGraphFactoryDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None, pre_filter=None):
        super().__init__(root, transform, pre_transform, pre_filter)

        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_dir(self):
        return "./"

    @property
    def processed_dir(self):
        return os.path.join(self.root, "train_processed")

    @property
    def raw_file_names(self):
        return ['train.csv']

    @property
    def processed_file_names(self):
        return ['train-graph.pt']

    def download(self):
        pass

    def process(self):

        data_list = self.read_data()

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]

        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

    def read_data(self):
        df_train = pd.read_csv(self.raw_paths[0])
        all_data = []
        for i in tqdm(range(df_train.shape[0])):
            text = df_train.text[i].lower().replace("\n", "").strip()
            doc = tokenizer.text_to_doc(text)
            input_ids = tokenizer.doc_to_sequence(doc)
            label = df_train.loc[i, ["target"]].to_list()

            x = torch.tensor(input_ids.reshape((-1, 1)), dtype=torch.int32)
            edge_index  = torch.tensor(train_idx2graph[i], dtype=torch.long)
            y = torch.tensor(np.array(label), dtype=torch.float32).reshape(-1, 1)
            data = Data(x=x, edge_index=edge_index, y=y)

            all_data.append(data)
        return all_data

In [30]:
dataset = TrainGraphFactoryDataset(root='./')

In [31]:
print(dataset.raw_paths, dataset.processed_paths)

['./train.csv'] ['./train_processed/train-graph.pt']


In [32]:
dataset.data

/home/blackswan/anaconda3/lib/python3.11/site-packages/torch_geometric/data/in_memory_dataset.py:301: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


Data(x=[316326505, 1], edge_index=[2, 628944656], y=[434666, 1])

In [33]:
dataset[2]

Data(x=[209, 1], edge_index=[2, 396], y=[1, 1])

In [34]:
dataset[200000].y

tensor([[0.]])

In [35]:
len(dataset)

434666

In [36]:
dataset = dataset.shuffle()
ratio_cut= 0.8
train_len = int(ratio_cut*len(dataset))
train_dataset= dataset[:train_len]
val_dataset= dataset[train_len:]
print(len(train_dataset),len(val_dataset))


347732 86934


In [37]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.optim import lr_scheduler
from torch_geometric.data import Data
from torch_geometric.data import InMemoryDataset
from torch_geometric.nn import GCNConv, SAGEConv, GATConv, TransformerConv, GATv2Conv, ChebConv, ResGatedGraphConv
import torch_geometric.nn as pyg_nn
from torch_geometric.loader import DataLoader

In [38]:
from torch_geometric.data import DataLoader

train_loader = DataLoader(train_dataset,batch_size=32)
val_loader = DataLoader(val_dataset, batch_size=32)

/home/blackswan/anaconda3/lib/python3.11/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [39]:
print(embedding_matrix)

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000

In [40]:
valid = []
invalid = []
for i, data in tqdm(enumerate(train_loader)):
    try:
        if data.num_nodes > data.edge_index.max():
            valid.append(data)
        else:
            invalid.append(i)
    except ValueError as ie:
        print('error in train edge index')


val_valid = []
val_invalid = []
for i, data in tqdm(enumerate(val_loader)):
    try:
        if data.num_nodes>data.edge_index.max():
            val_valid.append(data)
        else:
            val_invalid.append(i)
    except IndexError as ie:
        print('error in val edge index')

10867it [03:09, 57.25it/s]
2717it [00:45, 59.14it/s]


In [41]:
(len(valid)/len(train_loader))*100

48.651881844115216

In [42]:
(len(invalid)/len(train_loader))*100

51.34811815588479

In [43]:
(len(val_valid)/len(val_loader))*100

48.141332351858665

In [44]:
(len(val_invalid)/len(val_loader))*100

51.858667648141335

In [53]:
from torch_geometric.data import Batch
from torch_geometric.nn import GraphSAGE


class FeedbackModel(nn.Module):
    def __init__(self, embedding_matrix):

        super(FeedbackModel, self).__init__()

        self.embed = nn.Embedding.from_pretrained(torch.tensor(embedding_matrix, dtype=torch.float), freeze=False)
        # GCNConv SAGEConv ResGatedGraphConv GraphConv(300, 128) 
        # TransformerConv GATv2Conv GATConv(300, 128, heads=4) ChebConv(300, 128, K=2)
        # GCNConv SAGEConv ResGatedGraphConv GraphConv(128, 64) 
        # TransformerConv  GATv2Conv GATConv(4*128, 64) ChebConv(128, 64, K=2)
        #         self.gru = nn.GRU(256, 256, num_layers=1, 
        #                           dropout=0, batch_first=True,
        #                           bidirectional=False)          # RNN, GRU
        # output: (N, L, D∗Hout), D = 2 if bidirectional=True otherwise 1
        # h_n: (D∗num_layers, N, Hout)
        self.gc1   = GATv2Conv(300, 128, heads=4)
        #self.gc1   = GraphSAGE (300,128, 32)
        self.pool1 = pyg_nn.TopKPooling(128, ratio=0.8)
        self.gc2   = GCNConv(128, 64)
        self.pool2 = pyg_nn.TopKPooling(64, ratio=0.8)
        self.lin1  = nn.Linear(64, 32)
        self.lin2  = nn.Linear(32, 1)

    def forward(self, data):
        data_list = [Data(x=x_, edge_index=data.edge_index, sample_batched=data.batch) for x_ in (data.x)]
        batch = Batch.from_data_list(data_list)
        print('(Data List Loaded)')
        x = batch.x
        sample_batch=batch.sample_batched
        print('(squeeze passed)')
        #x = torch.clamp(x, 0, self.embed.num_embeddings - 1)
        print('(embed 1 pre-Loaded')
        x = self.embed(x)
    
        print('(Layer 1 pre-Loaded)')
        x = F.relu(self.gc1(x, edge_index=batch.edge_index))
        print('(Layer 1 Loaded)')
        x, edge_index, edge_attr, batch2, perm, score = self.pool1(x, batch.edge_index, None, sample_batch)
        x1 = torch.cat([pyg_nn.global_max_pool(x, batch2), pyg_nn.global_mean_pool(x, batch2)], dim=1)

        x = F.relu(self.gc2(x, batch.edge_index))
        print('(Layer 2 Loaded)')
        x, edge_index, edge_attr, batch2, perm, score = self.pool2(x, data.edge_index, None, sample_batch)
        x2 = torch.cat([pyg_nn.global_max_pool(x, batch2), pyg_nn.global_mean_pool(x, batch2)], dim=1)

        x = x1 + x2
        x = F.relu(self.lin1(x))
        print('(Layer 3 Loaded)')
        x = F.dropout(x, p=0.5, training=self.training)
        output = F.relu(self.lin2(x))
        return output



In [54]:
model = FeedbackModel(embedding_matrix)
model

FeedbackModel(
  (embed): Embedding(2, 300)
  (gc1): GATv2Conv(300, 128, heads=4)
  (pool1): TopKPooling(128, ratio=0.8, multiplier=1.0)
  (gc2): GCNConv(128, 64)
  (pool2): TopKPooling(64, ratio=0.8, multiplier=1.0)
  (lin1): Linear(in_features=64, out_features=32, bias=True)
  (lin2): Linear(in_features=32, out_features=1, bias=True)
)

In [55]:
epochs = 60

criterion = nn.MSELoss()
optimizer = Adam(model.parameters(), lr=1e-5)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=6)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
criterion.to(device)

MSELoss()

In [52]:



total_loss = []
for epoch_num in range(epochs):

    model.train()
    total_loss_train = 0
    print(f'epoch: {epoch_num} in progress...')
    print(f'train computing')
    
    for sample_batched in valid:
        print('in train loop')
        sample_batched = sample_batched.to(device)
        optimizer.zero_grad()
        outputs = model(sample_batched)
        print('outputs loaded')
        label = sample_batched.y.to(device)
        loss = criterion(outputs, label)
        loss.backward()
        total_loss_train += loss.item()
        optimizer.step()
        

    model.eval()
    total_loss_val = 0
    
    print(f'train evaluating')
    with torch.no_grad():
        for sample_batched in val_valid:
                sample_batched = sample_batched.to(device)
                outputs = model(sample_batched)
                label = sample_batched.y.to(device)
                loss = criterion(outputs, label)
                total_loss_val += loss.item()

    scheduler.step(total_loss_val / len(val_dataset))

    print(f'Epoch: %02.0f ended | Train Loss: {total_loss_train / len(train_dataset): .3f} | Val Loss: {total_loss_val / len(val_dataset): .3f}' % (epoch_num + 1))
    total_loss.append([total_loss_train / len(train_dataset), total_loss_val / len(val_dataset)])



epoch: 0 in progress...
train computing
in train loop
(Data List Loaded)


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)